<a href="https://colab.research.google.com/github/jnqeras/ARC/blob/master/tesis4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Descripción

En esta notebook voy a hacer lo mimso que en tesis2.ipynb, pero generando los pasándole una lista de strings a model.encode(), como lo indica la documentación de model.encode().
Lo voy a hacer así, porque si lo hago así, model.encode() devuelve un tensor con los embbedings, que es literalmente lo que toma la función util.tensor().

Si generase los embeddings con un for loop, como lo hacía en tesis3.ipynb, después tengo que transofrmar la lista de embeddings en un tensor y las dimensiones quedan raras y no se le pueden pasar a util.cos_sim.

No voy a chequear si los embeddings generados de a uno son los mismos que los generados con la lista, voy a asumir que esto es así.

# **Set-up**



Installation
We recommend Python 3.6 or higher, PyTorch 1.6.0 or higher and transformers v4.6.0 or higher. The code does not work with Python 2.7.

Install SentenceTransformers

New section
Install with pip

Install the sentence-transformers with pip:

In [1]:
# Fixme: Para correr el script localmente, comentar esta línea (para correr en Colab, descomentar):
!pip install -U sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 2.6 MB/s 
     |████████████████████████████████| 5.5 MB 11.3 MB/s 
     |████████████████████████████████| 1.3 MB 49.7 MB/s 
     |████████████████████████████████| 163 kB 53.5 MB/s 
     |████████████████████████████████| 7.6 MB 30.0 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=15926f78ca43403c08ee154851d5589bbb343fafeab7a7e81beb57db5784de2a
  Stored in directory: /root/.cache/pip/wheels/bf/06/fb/d59c1e5bd1dac7f6cf61ec0036cc3a10ab8fecaa6b2c3d3ee9
Successfully built sentence-transformers


# **Trabajando con el Dataset de CONAN:**


In [2]:
import pandas as pd     #Fixme: cuando esté todo terminado, puedo mover todos los imports a la parte superior.
import numpy as np

url = 'https://raw.githubusercontent.com/jnqeras/CONANDatasetCopy/main/CONAN/CONAN.csv'

df1 = pd.read_csv(url)

# Dataset is now stored in a Pandas Dataframe

In [3]:
# Observo los primeros cinco elementos del Dataframe.
df1.head()

,cn_id,hateSpeech,counterSpeech,hsType,hsSubType,cnType,age,gender,educationLevel
0,ENT1ST0001HS0033CN000021,"According to a recent OFSTED report, a school ...","To be fair, the OFSTED report is more concerne...",Islamophobia,crimes,facts,61.0,male,Bachelor
1,ENT1ST0001HS0033CN000021P1,In Birmingham there is a school where girls an...,"To be fair, the OFSTED report is more concerne...",Islamophobia,crimes,facts,61.0,male,Bachelor
2,ENT1ST0001HS0033CN000021P2,A school in Birningham is still separating gir...,"To be fair, the OFSTED report is more concerne...",Islamophobia,crimes,facts,61.0,male,Bachelor
3,ENT1ST0001HS0033CN000022,"According to a recent OFSTED report, a school ...",While this story is presented as an 'Islamic i...,Islamophobia,crimes,facts,61.0,male,Bachelor
4,ENT1ST0001HS0033CN000022P1,In Birmingham there is a school where girls an...,While this story is presented as an 'Islamic i...,Islamophobia,crimes,facts,61.0,male,Bachelor


In [4]:
# Selecciono las dos columnas que me interesan
sentences_conan = df1[['hateSpeech', 'counterSpeech']]

sentences_conan[0:1000]

,hateSpeech,counterSpeech
0,"According to a recent OFSTED report, a school ...","To be fair, the OFSTED report is more concerne..."
1,In Birmingham there is a school where girls an...,"To be fair, the OFSTED report is more concerne..."
2,A school in Birningham is still separating gir...,"To be fair, the OFSTED report is more concerne..."
3,"According to a recent OFSTED report, a school ...",While this story is presented as an 'Islamic i...
4,In Birmingham there is a school where girls an...,While this story is presented as an 'Islamic i...
...,...,...
995,Muslims do not have nothing useful that can en...,Muslims have been part of the British fabric s...
996,Muslims have nothing to contribute to our soci...,Do you know that you are using a lot of things...
997,Muslims cannot add any value to our culture.,Do you know that you are using a lot of things...
998,Muslims do not have nothing useful that can en...,Do you know that you are using a lot of things...


Hago una selección de únicamemente las contranarrativas (para armar el ránking que consiste en contranarrativas exclusivamente).

In [5]:
# Selecciono la columna de contranarrativas
counternarratives_conan = df1[['counterSpeech']]

counternarratives_conan[0:1000]

,counterSpeech
0,"To be fair, the OFSTED report is more concerne..."
1,"To be fair, the OFSTED report is more concerne..."
2,"To be fair, the OFSTED report is more concerne..."
3,While this story is presented as an 'Islamic i...
4,While this story is presented as an 'Islamic i...
...,...
995,Muslims have been part of the British fabric s...
996,Do you know that you are using a lot of things...
997,Do you know that you are using a lot of things...
998,Do you know that you are using a lot of things...


In [6]:
#sólo para hacer un sanity check, voy a seleccionar los discursos de odio de CONAN, eliminar los repetidos 
# y aseurarme que se cumple sentences_conan_list_sin_repetidos = counternarratives_conan_list_sin_repetidos + hate_speech_conan_list_sin_repetidos
# Selecciono las dos columnas que me interesan
hate_speech_conan = df1[['hateSpeech']]

hate_speech_conan [0:1000]

,hateSpeech
0,"According to a recent OFSTED report, a school ..."
1,In Birmingham there is a school where girls an...
2,A school in Birningham is still separating gir...
3,"According to a recent OFSTED report, a school ..."
4,In Birmingham there is a school where girls an...
...,...
995,Muslims do not have nothing useful that can en...
996,Muslims have nothing to contribute to our soci...
997,Muslims cannot add any value to our culture.
998,Muslims do not have nothing useful that can en...


Creo embeddings para los mensajes de odio y sus contranarrativas.

In [7]:
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('all-MiniLM-L6-v2')         
#fixme: acá está la documentación de este modelo, si cambio de modelo, cambiará el formato en el que tengo que pasarle las frases que quiero encodear (tuits odio y contranarrativas).

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/350 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/349 [00:00<?, ?B/s]

*Convierto* los dataframes en dos listas de strings (porque es lo que toma el modelo 'all-MiniLM-L6-v2'). Voy a eliminar los repetidos.


In [8]:
# convierto el dataframe en una lista sin repetidos
sentences_conan_list_sin_repetidos =list(set(list(sentences_conan.values.flatten()))) #fixme: mega importante: el set está para eliminar repetidos de la lista. Set toma una lista y devuleve un set (creo), posteriormente vuelvo a aplicar list asegurarme que sentences_conan_list_sin_repetidos es una lista.

#print(sentences_conan_list_sin_repetidos[0:5])

['False, Islam is a religion that accepts all religions and respects them.', 'It does not disturb anything.', 'Someone needs to put those Muslims in a hole in the ground before they start to blow us all up again.', "Chrétiens, partez au Vatican. Juifs, allez en Palestine. Bouddhistes partez pour l'Inde! Chacun doit pouvoir vivre sa religion là où il est né.", "I am not sure I would use 'invasion' to describe the spead of a particular faith or belief, but isnt' that what religions do. Spead; by text, by preaching, by word of mouth, by proselytisation?"]


In [9]:
counternarratives_conan_list_sin_repetidos =list(set(list(counternarratives_conan.values.flatten()))) #fixme: mega importante: el set está para eliminar repetidos de la lista. Set toma una lista y devuleve un set (creo), posteriormente vuelvo a aplicar list asegurarme que counternarratives_conan_list_sin_repetidos es una lista.

#print(counternarratives_conan_list_sin_repetidos[0:5])

['False, Islam is a religion that accepts all religions and respects them.', 'It does not disturb anything.', "Chrétiens, partez au Vatican. Juifs, allez en Palestine. Bouddhistes partez pour l'Inde! Chacun doit pouvoir vivre sa religion là où il est né.", "I am not sure I would use 'invasion' to describe the spead of a particular faith or belief, but isnt' that what religions do. Spead; by text, by preaching, by word of mouth, by proselytisation?", "Nous n'obligeons personne à manger hallal."]


In [10]:
hate_speech_conan_list_sin_repetidos =list(set(list(hate_speech_conan.values.flatten()))) #fixme: mega importante: el set está para eliminar repetidos de la lista. Set toma una lista y devuleve un set (creo), posteriormente vuelvo a aplicar list asegurarme que hate_speech_conan_list_sin_repetidos es una lista.

#print(counternarratives_conan_list_sin_repetidos[0:5])


['False, Islam is a religion that accepts all religions and respects them.', 'It does not disturb anything.', "Chrétiens, partez au Vatican. Juifs, allez en Palestine. Bouddhistes partez pour l'Inde! Chacun doit pouvoir vivre sa religion là où il est né.", "I am not sure I would use 'invasion' to describe the spead of a particular faith or belief, but isnt' that what religions do. Spead; by text, by preaching, by word of mouth, by proselytisation?", "Nous n'obligeons personne à manger hallal."]


Fixme: acá podría chequear si sentences_conan_list_sin_repetidos y counternarratives_conan_list_sin_repetidos efectivamente no tienen repetidos.

In [11]:
#Veo cuántos elementos tiene sentences_conan_list eliminado repetidos.
len(sentences_conan_list_sin_repetidos)

7659

In [12]:
#Veo cuántos elementos tiene counternarratives_conan_list eliminado repetidos.
len(counternarratives_conan_list_sin_repetidos)

6803

In [13]:
#Veo cuántos elementos tiene hate_speech_conan_list_sin_repetidos eliminado repetidos.
len(hate_speech_conan_list_sin_repetidos)

856

In [14]:
# Chequeo si da la suma:
len(sentences_conan_list_sin_repetidos) == len(counternarratives_conan_list_sin_repetidos) + len(hate_speech_conan_list_sin_repetidos)

True

#Genero los embeddings de los discursos de odio pasándole la lista de discursos de odio sin repetidos (fixme: esto lo hago, porque si gerero los embeddings así, me devuelve un tensor con todos los embeddings, que es lo que posteriormente voy a necesitar para calcular la cosine similarty).

In [ ]:
print('Generando Embeddings para los discursos de odio.')

In [15]:
embeddings_hate_speech_conan_list_sin_repetidos_creado_por_lista = model.encode(hate_speech_conan_list_sin_repetidos);

#Genero los embeddings para las contranarrativas pasándole la lista de contranarrativas sin repetidos (fixme: esto lo hago, porque si gerero los embeddings así, me devuelve un tensor con todos los embeddings, que es lo que posteriormente voy a necesitar para calcular la cosine similarty).

In [ ]:
print('Generando Embeddings para las contranarrativas.')

In [16]:
embeddings_counternarratives_conan_list_sin_repetidos_creado_por_lista = model.encode(counternarratives_conan_list_sin_repetidos);

# **Ploteo de Embeddings.**

Fixme: la sugerencia de Lau y Dami es graficar usando PCA o T-SNE

Intentar plotear como en esta página: https://inside-machinelearning.com/en/efficient-sentences-embedding-visualization-tsne/

Este ploteo está bueno: https://github.com/ashutoshsingh25/Plotting-multidimensional-vectors-using-t-SNE/blob/master/TSNE%20Code%20for%20clusring%20image%20and%20text%20vectors%20with%20labels.ipynb

Este ploteo está bueno: https://towardsdatascience.com/plotting-text-and-image-vectors-using-t-sne-d0e43e55d89

k-Means: para encontrar el valor óptimo de k ver Gap Statistic: https://hastie.su.domains/Papers/gap.pdf

Si no, chusmear: Cluster: https://towardsdatascience.com/plotting-text-and-image-vectors-using-t-sne-d0e43e55d89
    


#**Prueba de hipótesis a mano: un embedding calculado como propone la hipótesis comparado contra todo el dataset**

Voy a hacer una prueba a mano de la versión del experimento que es más fácil de implementar, es decir:
1. Partir del embedding de contranarrativa_1.
2. Al embedding de contranarrativa_1, restarle el embedding de odio_1. Suponiendo que esto nos deja en el espacio vectorial cercano a "contranarrativa sin mensaje de odio asignado".
3. Sumarle el embedding de odio_2 y esperar que esto nos deje en el espacio vectorial cercano al embedding de contranarrativa_2.



El plan es:

1. Voy a elegir una contranarrativa_1 a mano y voy a buscar su sentence embedding.
2. Voy a elegir odio_1 (el mensaje de odio que le corresponde a contranarrativa_1 en el dataset del CONAN), voy a buscar su sentence embedding y restarselo a embedding de contranarrativa_1.
3. Voy a elegir un mensaje odio_2, voy a buscar su sentence embedding y voy a sumárselo al resultado del paso 2.
4. Voy a calcular la similaridad coseno entre el resultado del paso anterior y los embeddings de todas las **contranarratiavs del Conan**. Espero que la que tenga valor más alto, corresponda a una contranarrativa que "sirva" (en sentido de que contraargumente) contra al mensaje odio_2, idealmente quiero que sea una contranarrativa que en el dataset original (Conan), aparezca asignada a odio_2. 

## **Nota al márgen: Cómo elegir pares de contranarrativas y discursos de odio para las pruebas a mano**

Se puede elegir un odio_2 tal que sea muy diferente a odio_1 (también se puede elegir un odio_2 tal que sea muy similar a odio_1), además hay discursos de odio a los que se responde con distintas contranarrativas, puedo elegir odio_1=odio_2 y contranarrativa_1 != contranarrativa_2. También los puedo elegir según su similitud coseno (similitud coseno entre contranarrativa_1 y contranarrativa_2 y entre odio_1 y odio_2).

## **Prueba a mano n1:**

En esta primera prueba, voy a elegir los mensaje de odio y sus contranarrativas sin pensar demasiado cómo los elijo.

Elijo:

* contranarrativa_1: To be fair, the OFSTED report is more concerned with lack of enforcement and less about focussing on the practice of any particular faith.

El mensaje de odio que le corresponde es:
* odio_1: 	According to a recent OFSTED report, a school in Birmingham is still segregating girls and boys, despite a Hight Court ruling in 2017 that this is unlawful.


* contranarrativa_2: While this story is presented as an 'Islamic issue', the OFSTED report refers to 'countless' faith schools still that still illegally maintain gender segregation - my guess is this is a cross-faith practice and not exclusive to Islamic faith schools.

El mensaje de odio que le corresponde es:

* odio_2: In Birmingham there is a school where girls and boys are separeted even if it isn't legal.	


#### **Ejecución del plan:**

##### 1. Voy a elegir una contranarrativa_1 a mano y voy a buscar su sentence embedding

* contranarrativa_1: To be fair, the OFSTED report is more concerned with lack of enforcement and less about focussing on the practice of any particular faith.

In [17]:
#Busco el índice de contranarrativa_1.
i = counternarratives_conan_list_sin_repetidos.index('To be fair, the OFSTED report is more concerned with lack of enforcement and less about focussing on the practice of any particular faith.')

In [18]:
#Chequeo que efectivamente el índice obtenido corresponda a contranarrativa_1 (si es así, esta celda debe devolver true):
counternarratives_conan_list_sin_repetidos[i] == 'To be fair, the OFSTED report is more concerned with lack of enforcement and less about focussing on the practice of any particular faith.'

True

In [19]:
contranarrativa_1_embedding_creado_por_lista = embeddings_counternarratives_conan_list_sin_repetidos_creado_por_lista[i];

In [20]:
# Comento esta celda, porque como no está dando el mismo embedding, no quiero por error utilizar embedding_contranarrativa_1
#Genero el embedding para la contranarrativa_1 
#embedding_contranarrativa_1 = model.encode(counternarratives_conan_list_sin_repetidos[i])

In [21]:
# Comento esta celda, porque como no está dando el mismo embedding, no quiero por error utilizar embedding_contranarrativa_1

#Chequeo si el embedding_contranarrativa_1 es igual al embedding que cree con la lista más arriba (esta 
# celda no hace al experimento todavía, pero me sirve para saber que puedo calcular los embeddings de varias
# maneras). Si los embeddings son iguales, esta celda debe devolver true:
#np.array_equal(embedding_contranarrativa_1[0], embeddings_counternarratives_conan_list_sin_repetidos_creado_por_lista[i])

##### 2. Voy a elegir odio_1 (el mensaje de odio que le corresponde a contranarrativa_1 en el dataset del CONAN), voy a generar su sentence embedding y restarselo a embedding de contranarrativa_1.



Se que odio_1 es el siguiente, porque lo encuentro revisando el dataframe del conan:

* odio_1: According to a recent OFSTED report, a school in Birmingham is still segregating girls and boys, despite a Hight Court ruling in 2017 that this is unlawful.

In [22]:
# Busco el índice de odio_1.
j = hate_speech_conan_list_sin_repetidos.index('According to a recent OFSTED report, a school in Birmingham is still segregating girls and boys, despite a Hight Court ruling in 2017 that this is unlawful.')

In [23]:
#Chequeo que efectivamente el índice obtenido corresponda a odio_1  (si es así, esta celda debe devolver true):
hate_speech_conan_list_sin_repetidos[j] == 'According to a recent OFSTED report, a school in Birmingham is still segregating girls and boys, despite a Hight Court ruling in 2017 that this is unlawful.'

True

In [24]:
odio_1_embedding_creado_por_lista = embeddings_hate_speech_conan_list_sin_repetidos_creado_por_lista[j];

In [25]:
# Comento esta celda, porque como no está dando el mismo embedding, no quiero por error utilizar embedding_odio_1

#Genero el embedding para odio_1
#embedding_odio_1 = model.encode(hate_speech_conan_list_sin_repetidos[j])

In [26]:
# Comento esta celda, porque como no está dando el mismo embedding, no quiero por error utilizar embedding_odio_1

#Chequeo si el embedding_odio_1 es igual al embedding que cree con un for loop más arriba (esta celda no
# hace al experimento todavía, pero me sirve para saber que puedo calcular los embeddings de varias maneras):
#np.array_equal(embedding_odio_1, embeddings_hate_speech_conan_list_sin_repetidos_creado_por_lista[j])

In [27]:
# Resto el embedding de odio_1 al de contranarrativa_1.
embedding_contranarrativa_1_sin_discurso_de_odio_1 = contranarrativa_1_embedding_creado_por_lista - odio_1_embedding_creado_por_lista

##### 3. Voy a elegir un mensaje odio_2, voy a buscar su sentence embedding y voy a sumárselo al resultado del paso 2.

* odio_2: In Birmingham there is a school where girls and boys are separeted even if it isn't legal.

In [28]:
# Busco el índice de odio_2 para obtener su sentence embedding.
k = hate_speech_conan_list_sin_repetidos.index('In Birmingham there is a school where girls and boys are separeted even if it isn\'t legal.')

In [29]:
#Chequeo que efectivamente el índice obtenido corresponda a odio_2  (si es así, esta celda debe devolver true):
hate_speech_conan_list_sin_repetidos[k] == 'In Birmingham there is a school where girls and boys are separeted even if it isn\'t legal.'

True

In [30]:
odio_2_embedding_creado_por_lista = embeddings_hate_speech_conan_list_sin_repetidos_creado_por_lista[k];

In [31]:
# Comento esta celda, porque como no está dando el mismo embedding, no quiero por error utilizar embedding_odio_2

#Genero el embedding para odio_2
#embedding_odio_2 = model.encode([hate_speech_conan_list_sin_repetidos[k]])

In [32]:
# Comento esta celda, porque como no está dando el mismo embedding, no quiero por error utilizar embedding_odio_2

#Chequeo si el embedding_odio_2 es igual al embedding que cree con un for loop más arriba (esta celda no
# hace al experimento todavía, pero me sirve para saber que puedo calcular los embeddings de varias maneras):
#np.array_equal(embedding_odio_2, embeddings_hate_speech_conan_list_sin_repetidos_creado_por_lista[k])

In [33]:
# Sumo el embedding de odio_2 al resultado del paso 2 (embedding_contranarrativa_1_sin_discurso_de_odio_1).
embedding_cercano_a_contranarrativa_para_odio_2 = embedding_contranarrativa_1_sin_discurso_de_odio_1 + odio_2_embedding_creado_por_lista

In [34]:
# chequeo que el embedding embedding_cercano_a_contranarrativa_para_odio_2 tenga el mismo shape que los
# embeddings de embeddings_hate_speech_conan_list_sin_repetidos_creado_con_for y que el emmeding
# embedding_contranarrativa_1
embedding_cercano_a_contranarrativa_para_odio_2.shape == embeddings_counternarratives_conan_list_sin_repetidos_creado_por_lista[0].shape == contranarrativa_1_embedding_creado_por_lista.shape

True

##### 4. Voy a calcular la similaridad coseno entre el resultado del paso anterior (embedding_cercano_a_contranarrativa_para_odio_2) y los embeddings de todas las contranarrativas del Conan (embeddings_counternarratives_conan_list_sin_repetidos_creado_por_lista). 

Espero que la que tenga valor más alto, corresponda a una contranarrativa que "sirva" (en sentido de que contraargumente) contra al mensaje odio_2, idealmente quiero que sea una contranarrativa que en el dataset original (Conan), aparezca asignada a odio_2.

La contranarrativa correspondiente a odio_2 en el dataset es: 
contranarrativa_2: While this story is presented as an 'Islamic issue', the OFSTED report refers to 'countless' faith schools still that still illegally maintain gender segregation - my guess is this is a cross-faith practice and not exclusive to Islamic faith schools.

Como mejor caso, quisiera que el embedding con similitud coseno más grande con embedding_cercano_a_contranarrativa_para_odio_2, sea el embedding de contranarrativa_2.

Antes de calcular la similaridad coseno entre embedding_cercano_a_contranarrativa_para_odio_2 y embeddings_counternarratives_conan_list_sin_repetidos_creado_con_for, voy a examinarlos para estar seguro de que son comparables.

Puedo ver que uno es una lista embeddings y el otro es un embedding.

In [35]:
  # Computo la similaridad coseno de embedding_cercano_a_contranarrativa_para_odio_2, 
  #contra todos los embeddings de todas las contranarrativas del Conan.

# Calculo la similaridad coseno 
cos_sim_embedding_cercano_a_contranarrativa_para_odio_2_conan = util.cos_sim(embedding_cercano_a_contranarrativa_para_odio_2, embeddings_counternarratives_conan_list_sin_repetidos_creado_por_lista)

In [36]:
cos_sim_embedding_cercano_a_contranarrativa_para_odio_2_conan.shape

torch.Size([1, 6803])

In [37]:
len(cos_sim_embedding_cercano_a_contranarrativa_para_odio_2_conan[0])

6803

In [38]:
cos_sim_embedding_cercano_a_contranarrativa_para_odio_2_conan

tensor([[ 0.1761,  0.1118,  0.0447,  ...,  0.1924,  0.0723, -0.0151]])

In [39]:
# Chequeo que tengo una distancia coseno por cada elemento en counternarratives_conan_list_sin_repetidos (si es así esta celda debería devolver true).
len(counternarratives_conan_list_sin_repetidos) == len(cos_sim_embedding_cercano_a_contranarrativa_para_odio_2_conan[0])

True

In [40]:
#Add all pairs to a list with their cosine similarity score
all_sentence_combinations_cercano_a_contranarrativa_para_odio_2 = []
for i in range(0, len(cos_sim_embedding_cercano_a_contranarrativa_para_odio_2_conan[0])):  # Fixme: mega importante: lo siguiente aplica para cuando estoy comparando un embedding contra todo el resto, cuando comparo todos contra todos, tengo que poner cos_sim, en vez de cos_sim[0]. Es importante pedirle la primera posición a cos_sim porque cos_sim es un tensor (una matriz, de 1xCantidadDeEmbedings contra los que estoy comparando al embedding i, por eso, si len(cos_sim[0])=1 y len(cos_sim[0])=CantidadDeEmbedings).
    all_sentence_combinations_cercano_a_contranarrativa_para_odio_2.append([cos_sim_embedding_cercano_a_contranarrativa_para_odio_2_conan[0][i], i]) #fixme: mega importante, acá a cos_sim_embedding_cercano_a_contranarrativa_para_odio_2_conan le hadrcodeo un [0], porque este es un tensor de tamaño 1x29976, si quiero acceder a la primera "fila", necesito ese [0]. Quizás haya una forma más elegante de resolverlo.

In [41]:
all_sentence_combinations_cercano_a_contranarrativa_para_odio_2;

## Métricas a mano para la prueba mano n1:

Busco la forma de dado un tuit de odio, obtener todas sus contranarrativas en Conan. Lo resuelvo así:

In [103]:
df_contranarrativas_en_conan_para_odio_2=sentences_conan.loc[sentences_conan['hateSpeech'] == 'In Birmingham there is a school where girls and boys are separeted even if it isn\'t legal.', 'counterSpeech']
df_contranarrativas_en_conan_para_odio_2

1     To be fair, the OFSTED report is more concerne...
4     While this story is presented as an 'Islamic i...
7     The quoted report refers to 'countless faith s...
10    Gender segregation in faith schools is clearly...
Name: counterSpeech, dtype: object

Busco forma de chequear si una frase está en un dataframe:

In [44]:
'To be fair, the OFSTED report is more concerned with lack of enforcement and less about focussing on the practice of any particular faith.' in df_contranarrativas_en_conan_para_odio_2.values

True

### Métrica 1: cuántos de las top 10 contranarrativas sugeridas en el ranking, efectivamente aparecen en el Conan como contranarrativas para el discurso de odio_2

In [45]:
metrica1 = 0;
for i in range(0,len(counternarratives_ranking_list)):
  if counternarratives_ranking_list[i] in df_contranarrativas_en_conan_para_odio_2.values :
      metrica1 += 1;
print('En la prueba a mano el ranking eligió',metrica1, 'de las', df_contranarrativas_en_conan_para_odio_2.shape[0],'contranarrativas que existen en el Conan')

El ranking eligió 2 de las 4 contranarrativas que existen en el Conan


Hago el chequeo a mano y corroboro que la métrica está bien.

# Parametrización de funciones de prueba de hipótesis a mano para probar en todos los casos:

## Hago una prueba a mano del cuerpo del for (para chequear que me da igual a la sección "Prueba a mano n1"):

In [46]:
#Busco el índice de contranarrativa_1.
i = counternarratives_conan_list_sin_repetidos.index('To be fair, the OFSTED report is more concerned with lack of enforcement and less about focussing on the practice of any particular faith.')

In [47]:
# La contranarrativa i es:
counternarratives_conan_list_sin_repetidos[i]

'To be fair, the OFSTED report is more concerned with lack of enforcement and less about focussing on the practice of any particular faith.'

In [48]:
#selecciono el embedding de la contranarrativa_i
contranarrativa_i_embedding_creado_por_lista = embeddings_counternarratives_conan_list_sin_repetidos_creado_por_lista[i];

In [49]:
#Chequeo si los siguientes dos embeddings son iguales (espero que lo sean):
np.array_equal(contranarrativa_i_embedding_creado_por_lista, contranarrativa_1_embedding_creado_por_lista)

True

In [108]:
# Busco los discursos de odio para la contranarrativa_i
df_oido_en_conan_para_contranarrativa_i = sentences_conan.loc[sentences_conan['counterSpeech'] == counternarratives_conan_list_sin_repetidos[i], 'hateSpeech']
df_oido_en_conan_para_contranarrativa_i

7319    Muslims are all scum.
Name: hateSpeech, dtype: object

In [51]:
# Elijo uno de los discursos de odio para la conranarrativa_i (la llamo odio_i). 
# Fixme: acá tengo más de una opción para elegir un discurso de odio para la 
# contranarrativa_i (tengo que pensar de que forma voy a elegir entre las diferentes opciones),
# acá estoy eligiendo el hatespeech 0.

odio_i = df_oido_en_conan_para_contranarrativa_i.values[0]
odio_i



'According to a recent OFSTED report, a school in Birmingham is still segregating girls and boys, despite a Hight Court ruling in 2017 that this is unlawful.'

In [52]:
# Busco el embedding de uno de los mensajes de odio de los que aparecen junto a contranarrativa_i en Conan 
# (fixme: para una contranarrativa puede haber varios mensajes de odio, decidir que hago todos ellos).

# Busco el índice de odio_i.
indice_odio_i = hate_speech_conan_list_sin_repetidos.index(odio_i)

# Busco el embedding para odio_i
odio_i_embedding_creado_por_lista = embeddings_hate_speech_conan_list_sin_repetidos_creado_por_lista[indice_odio_i]

In [53]:
#chequeo si odio_i == odio_1
hate_speech_conan_list_sin_repetidos[indice_odio_i] == hate_speech_conan_list_sin_repetidos[728]

False

In [54]:
#Chequeo si el embeddings de odio_i y de odio_1 son iguales (espero que lo sean):
np.array_equal(odio_i_embedding_creado_por_lista, odio_1_embedding_creado_por_lista)

True

In [55]:
# Resto el embedding de odio_i al de contranarrativa_i.
embedding_contranarrativa_i_sin_discurso_de_odio_i = contranarrativa_i_embedding_creado_por_lista - odio_i_embedding_creado_por_lista

In [56]:
#Chequeo si embedding_contranarrativa_i_sin_discurso_de_odio_i y embedding_contranarrativa_1_sin_discurso_de_odio_1 son iguales (espero que lo sean):
np.array_equal(embedding_contranarrativa_i_sin_discurso_de_odio_i, embedding_contranarrativa_1_sin_discurso_de_odio_1)


True

In [57]:
# Elijo un mensaje odio_k (distinto a odio_i)
# Fixme: acá puedo elegir cualquier mensaje de odio, quisiera que no sea odio_i, tengo que 
# eleigr una forma de elegir a odio_k
odio_k = hate_speech_conan_list_sin_repetidos[k]
odio_k

"In Birmingham there is a school where girls and boys are separeted even if it isn't legal."

In [58]:
# Busco el embedding de odio_k embedding
odio_k_embedding_creado_por_lista = embeddings_hate_speech_conan_list_sin_repetidos_creado_por_lista[k];

In [59]:
#Chequeo si el embeddings de odio_k y de odio_2 son iguales (espero que lo sean):
np.array_equal(odio_k_embedding_creado_por_lista, odio_2_embedding_creado_por_lista)

True

In [60]:
# Sumo el embedding de odio_K a embedding_contranarrativa_i_sin_discurso_de_odio_i.
embedding_cercano_a_contranarrativa_para_odio_k = embedding_contranarrativa_i_sin_discurso_de_odio_i + odio_k_embedding_creado_por_lista

In [61]:
#Chequeo si el embedding_cercano_a_contranarrativa_para_odio_k y embedding_cercano_a_contranarrativa_para_odio_2 son iguales (espero que lo sean):
np.array_equal(embedding_cercano_a_contranarrativa_para_odio_k, embedding_cercano_a_contranarrativa_para_odio_2)

True

In [110]:
# Calculo la similaridad coseno entre el resultado del paso anterior (embedding_cercano_a_contranarrativa_para_odio_k) y los embeddings de todas las contranarrativas del Conan (embeddings_counternarratives_conan_list_sin_repetidos_creado_por_lista).
cos_sim_embedding_cercano_a_contranarrativa_para_odio_k_conan = util.cos_sim(embedding_cercano_a_contranarrativa_para_odio_k, embeddings_counternarratives_conan_list_sin_repetidos_creado_por_lista)

#Add all pairs to a list with their cosine similarity score
all_sentence_combinations_cercano_a_contranarrativa_para_odio_k = []
for h in range(0, len(cos_sim_embedding_cercano_a_contranarrativa_para_odio_k_conan[0])):  # Fixme: mega importante: lo siguiente aplica para cuando estoy comparando un embedding contra todo el resto, cuando comparo todos contra todos, tengo que poner cos_sim, en vez de cos_sim[0]. Es importante pedirle la primera posición a cos_sim porque cos_sim es un tensor (una matriz, de 1xCantidadDeEmbedings contra los que estoy comparando al embedding h, por eso, si len(cos_sim[0])=1 y len(cos_sim[0])=CantidadDeEmbedings).
    all_sentence_combinations_cercano_a_contranarrativa_para_odio_k.append([cos_sim_embedding_cercano_a_contranarrativa_para_odio_k_conan[0][h], h]) #fixme: mega importante, acá a cos_sim_embedding_cercano_a_contranarrativa_para_odio_k_conan le hadrcodeo un [0], porque este es un tensor de tamaño 1x29976, si quiero acceder a la primera "fila", necesito ese [0]. Quizás haya una forma más elegante de resolverlo

#Sort list by the highest cosine similarity score
all_sentence_combinations_cercano_a_contranarrativa_para_odio_k = sorted(all_sentence_combinations_cercano_a_contranarrativa_para_odio_k, key=lambda x: x[0], reverse=True)


counternarratives_ranking_list =[]
for score, l in all_sentence_combinations_cercano_a_contranarrativa_para_odio_k[0:10]: #fixme: mega importante, chequear que el rango [0:10] son 10 resultados y no 11.
    counternarratives_ranking_list.append(counternarratives_conan_list_sin_repetidos[l])

df_contranarrativas_en_conan_para_odio_k = sentences_conan.loc[sentences_conan['hateSpeech'] == hate_speech_conan_list_sin_repetidos[k], 'counterSpeech'] #fixme: ojo con esto, tengo que chequear que el hate speech que me interesa es efectivamente "hate_speech_conan_list_sin_repetidos[k]"
print(df_contranarrativas_en_conan_para_odio_k)

metrica1 = 0;
for m in range(0,len(counternarratives_ranking_list)):
  if counternarratives_ranking_list[m] in df_contranarrativas_en_conan_para_odio_k.values :
      metrica1 += 1;
print('En la prueba a mano del cuerpo del for para el mensaje de odio número', k, 'el ranking eligió', metrica1, 'de las', df_contranarrativas_en_conan_para_odio_k.shape[0],'contranarrativas que existen en el Conan')

1596    I only wish followers of all religions and non...
1599    If that were true would not there only be one ...
1602    No it isn't and most 'tolerant' places in the ...
1605    The Christian world can be far from tolerant. ...
1608    Tolerance is not limited to one religion, or t...
1611    It is no more totalitarian than any other reli...
1614    Islam is belief system that contributed to pro...
1617    I am tired of the deliberate lies or appalling...
1620    The only thing that is ENOUGH is this discrimi...
1623    It sounds like you are saying that you have be...
1626    Oh the tolerant world that is spreading hate a...
Name: counterSpeech, dtype: object
En la prueba a mano del cuerpo del for para el mensaje de odio número 9 el ranking eligió 0 de las 11 contranarrativas que existen en el Conan


##Aplico la métrica 1 en parte el Dataset (para hacer un sanity check con parte del Dataset):

Fixme: importante: esta sección está comentada, porque usa los mismos nombres que la sección "Aplico la métrica 1 en todo el Dataset" y eso puede traer problemas. La comento y no la borro porque cada tanto me sirve tenerla a mano para hacer algunos sanity checks.

In [63]:
"""
#Chequeo cómo iterar en un dataframe:
for ind in df_oido_en_conan_para_contranarrativa_i.index:
    print(df_oido_en_conan_para_contranarrativa_i.loc[ind])
    print(ind)
"""

'\n#Chequeo cómo iterar en un dataframe:\nfor ind in df_oido_en_conan_para_contranarrativa_i.index:\n    print(df_oido_en_conan_para_contranarrativa_i.loc[ind])\n    print(ind)\n'

In [64]:
# Function returns N largest elements and the position they have in the input list.
def Nmaxelements(list1, N):
    final_list = []
    posicionesConMaximos = set()

    for i in range(0, N):
        max1 = 0

        for j in range(len(list1)):    
            if ((list1[j] >= max1) and (j not in posicionesConMaximos)):
                max1 = list1[j];
                indMax1 = j;
                
        posicionesConMaximos.add(indMax1)
        final_list.append((max1, indMax1))
         
    return(final_list)

###### Sanity checks para Nmaxelements

In [65]:
# Chequeo si Nmaxelements devuelve lo pedido.
# Driver code
list1 = [2, 6, 41, 85, 0, 3, 7, 6, 10]
N = 9
 
# Calling the function
Nmaxelements(list1, N)

[(85, 3), (41, 2), (10, 8), (7, 6), (6, 7), (6, 1), (3, 5), (2, 0), (0, 4)]

Chequeo si el ranking formado por Nmaxelements tiene los mismos elementos que el ranking que formaba ordenando todos los elementos. Efectivamente berifico que devuelven los mismos elementos.

In [66]:
listCos = cos_sim_embedding_cercano_a_contranarrativa_para_odio_k_conan[0].tolist()

In [67]:
print(Nmaxelements(listCos, 10))

[(0.8228143453598022, 3622), (0.44610679149627686, 3859), (0.43136531114578247, 4719), (0.42780253291130066, 4567), (0.4275047183036804, 6757), (0.4217926263809204, 4503), (0.41619032621383667, 2898), (0.3892439603805542, 4736), (0.3875080943107605, 1690), (0.38569727540016174, 1636)]


In [68]:
all_sentence_combinations_cercano_a_contranarrativa_para_odio_k[0:10]

[[tensor(0.8228), 3622],
 [tensor(0.4461), 3859],
 [tensor(0.4314), 4719],
 [tensor(0.4278), 4567],
 [tensor(0.4275), 6757],
 [tensor(0.4218), 4503],
 [tensor(0.4162), 2898],
 [tensor(0.3892), 4736],
 [tensor(0.3875), 1690],
 [tensor(0.3857), 1636]]

### Aplicación de métrica 1:

In [69]:
print('Calculando métrica 1')

Calculando métrica 1


In [102]:
# Hago el setup de algunos parámetros:
lista_pares_métrica_1_top10 = [] 
lista_pares_métrica_1_top5 = []
lista_pares_métrica_1_top3 = []
lista_pares_métrica_1_top1 = []
iteraciones = 0;

cantidad_contranarrativas_iteradas = 10   # junto a cantidad_discursos_odio_iterados determina la parte del dataset en la que se aplica la métrica 1.
cantidad_discursos_odio_iterados = 10     # fixme: el máximo es len(hate_speech_conan_list_sin_repetidos)     # junto a cantidad_contranarrativas_iteradas determina la parte del dataset en la que se aplica la métrica 1.

#fixme: partición for: este for iteraba 6803 veces
for i in range(0, cantidad_contranarrativas_iteradas): #fixme: lo que modifico es esto, antes decía: for i in range(0, len(counternarratives_conan_list_sin_repetidos)):
  # i será el índice de la contranarrativa_i.
  #selecciono el embedding de la contranarrativa_i
  contranarrativa_i_embedding_creado_por_lista = embeddings_counternarratives_conan_list_sin_repetidos_creado_por_lista[i];

  # Busco los discursos de odio para la contranarrativa_i
  df_oido_en_conan_para_contranarrativa_i = sentences_conan.loc[sentences_conan['counterSpeech'] == counternarratives_conan_list_sin_repetidos[i], 'hateSpeech']
  
  # Elijo cada uno de los discursos de odio que aparecen en Conan para la conranarrativa_i (los llamo odio_i). 
  #fixme: partición for: este for va variadno, pero más o menos itera 4 veces.
  for ind in  df_oido_en_conan_para_contranarrativa_i.index:
    odio_i = df_oido_en_conan_para_contranarrativa_i.loc[ind]

    # Busco el índice de odio_1 en hate_speech_conan_list_sin_repetidos.
    indice_odio_i = hate_speech_conan_list_sin_repetidos.index(odio_i)

    # Busco el embedding para odio_i  
    odio_i_embedding_creado_por_lista = embeddings_hate_speech_conan_list_sin_repetidos_creado_por_lista[indice_odio_i]
    
    # Resto el embedding de odio_i al de contranarrativa_i.
    embedding_contranarrativa_i_sin_discurso_de_odio_i = contranarrativa_i_embedding_creado_por_lista - odio_i_embedding_creado_por_lista
    
    #fixme: partición for: este for itera 856 veces.
    for k in range(0, cantidad_discursos_odio_iterados): # como máximo puede iterar hasta len(hate_speech_conan_list_sin_repetidos)
      if k == indice_odio_i: continue # Elijo un mensaje odio_k (distinto a odio_i)
      odio_k = hate_speech_conan_list_sin_repetidos[k]
      
      # Busco el embedding de odio_k embedding
      odio_k_embedding_creado_por_lista = embeddings_hate_speech_conan_list_sin_repetidos_creado_por_lista[k];

      # Sumo el embedding de odio_K a embedding_contranarrativa_i_sin_discurso_de_odio_i.
      embedding_cercano_a_contranarrativa_para_odio_k = embedding_contranarrativa_i_sin_discurso_de_odio_i + odio_k_embedding_creado_por_lista

      #fixme: chequear el cálculo decos_sim_embedding_cercano_a_contranarrativa_para_odio_k_conan, ver que devuelve util.cos_sim. 
      # Calculo la similaridad coseno entre el resultado del paso anterior (embedding_cercano_a_contranarrativa_para_odio_k) y los embeddings de todas las contranarrativas del Conan (embeddings_counternarratives_conan_list_sin_repetidos_creado_por_lista).
      #fixme: según entiendo, por la documentación de cos_sim, este tensor tiene en la segunda posición la cos_sim entre embedding_cercano_a_contranarrativa_para_odio_k y todos los embeddings de embeddings_counternarratives_conan_list_sin_repetidos_creado_por_lista. 
      cos_sim_embedding_cercano_a_contranarrativa_para_odio_k_conan = util.cos_sim(embedding_cercano_a_contranarrativa_para_odio_k, embeddings_counternarratives_conan_list_sin_repetidos_creado_por_lista)

      #transformo el tensor con las cos_sim en una lista con las cos_sim (quizás esta transformación podría evitarse, pero me parece más claro cómo eliminar un elemento de una lsita que de un tensor).
      lista_cos_sim_embedding_cercano_a_contranarrativa_para_odio_k_conan = cos_sim_embedding_cercano_a_contranarrativa_para_odio_k_conan[0].tolist()
      
      # Calculo la cos_sim e índice de las top 10 contranarrativas para odio k.
      ranking10MejoresContranarrativasParaOdioK = Nmaxelements(lista_cos_sim_embedding_cercano_a_contranarrativa_para_odio_k_conan, 10)

      # Calculo la cos_sim e índice de las top 5 contranarrativas para odio k.
      ranking5MejoresContranarrativasParaOdioK = ranking10MejoresContranarrativasParaOdioK[0:5] 

      # Calculo la cos_sim e índice de las top 3 contranarrativas para odio k.
      ranking3MejoresContranarrativasParaOdioK = ranking10MejoresContranarrativasParaOdioK[0:3]

      # Calculo la cos_sim e índice de la top 1 contranarrativa para odio k.
      ranking1MejoresContranarrativasParaOdioK = ranking10MejoresContranarrativasParaOdioK[0]

      # Busco las top 10 contranarrativas (busco efectivamente los strings, no los índices).
      counternarratives_ranking_list_top10 =[]
      for score, l in ranking10MejoresContranarrativasParaOdioK: #fixme: mega importante, chequear que el rango [0:10] son 10 resultados y no 11.
          counternarratives_ranking_list_top10.append(counternarratives_conan_list_sin_repetidos[l])

      # Busco las top 5 contranarrativas (busco efectivamente los strings, no los índices).
      counternarratives_ranking_list_top5 = counternarratives_ranking_list_top10[0:5]

      # Busco las top 3 contranarrativas (busco efectivamente los strings, no los índices).
      counternarratives_ranking_list_top3 = counternarratives_ranking_list_top10[0:3]

      # Busco la top 1 contranarrativa (busco efectivamente el string, no el índices).
      counternarratives_ranking_list_top1 = counternarratives_ranking_list_top10[0:1]

      #Fixme: acá puedo eliminar código repetido:
      # Calculo la métrica_1 para el caso contranarrativa_i, odio_i, odio_k y ranking de 10 contranarrativas 
      # y la guardo en la lista_pares_métrica_1_top10
      df_contranarrativas_en_conan_para_odio_k = sentences_conan.loc[sentences_conan['hateSpeech'] == odio_k, 'counterSpeech'] #fixme: ojo con esto, tengo que chequear que el hate speech que me interesa es efectivamente odio_k.
      metrica1 = 0;
      for m in range(0,len(counternarratives_ranking_list_top10)):
        if counternarratives_ranking_list_top10[m] in df_contranarrativas_en_conan_para_odio_k.values :
            metrica1 += 1;
      lista_pares_métrica_1_top10.append(('Contranarrativa_i, está en la posición', i,'en counternarratives_conan_list_sin_repetidos. Odio_i está en la posición numero', indice_odio_i, 'en hate_speech_conan_list_sin_repetidos.', 'Para el mensaje de odio en la posición', k,' en hate_speech_conan_list_sin_repetidos, el ranking de 10 contranarrativas contiene', metrica1, 'de las', df_contranarrativas_en_conan_para_odio_k.shape[0],'contranarrativas que existen en el conan para ese discurso de odio'))

      # Calculo la métrica_1 para el caso contranarrativa_i, odio_i, odio_k y ranking de 5 contranarrativas 
      # y la guardo en la lista_pares_métrica_1_top5
      df_contranarrativas_en_conan_para_odio_k = sentences_conan.loc[sentences_conan['hateSpeech'] == odio_k, 'counterSpeech'] #fixme: ojo con esto, tengo que chequear que el hate speech que me interesa es efectivamente odio_k.
      metrica1 = 0;
      for m in range(0,len(counternarratives_ranking_list_top5)):
        if counternarratives_ranking_list_top5[m] in df_contranarrativas_en_conan_para_odio_k.values :
            metrica1 += 1;
      lista_pares_métrica_1_top5.append(('Contranarrativa_i, está en la posición', i,'en counternarratives_conan_list_sin_repetidos. Odio_i está en la posición numero', indice_odio_i, 'en hate_speech_conan_list_sin_repetidos.', 'Para el mensaje de odio en la posición', k,' en hate_speech_conan_list_sin_repetidos, el ranking de 5 contranarrativas contiene', metrica1, 'de las', df_contranarrativas_en_conan_para_odio_k.shape[0],'contranarrativas que existen en el conan para ese discurso de odio'))

      # Calculo la métrica_1 para el caso contranarrativa_i, odio_i, odio_k y ranking de 3 contranarrativas 
      # y la guardo en la lista_pares_métrica_1_top3
      df_contranarrativas_en_conan_para_odio_k = sentences_conan.loc[sentences_conan['hateSpeech'] == odio_k, 'counterSpeech'] #fixme: ojo con esto, tengo que chequear que el hate speech que me interesa es efectivamente odio_k.
      metrica1 = 0;
      for m in range(0,len(counternarratives_ranking_list_top3)):
        if counternarratives_ranking_list_top3[m] in df_contranarrativas_en_conan_para_odio_k.values :
            metrica1 += 1;
      lista_pares_métrica_1_top3.append(('Contranarrativa_i, está en la posición', i,'en counternarratives_conan_list_sin_repetidos. Odio_i está en la posición numero', indice_odio_i, 'en hate_speech_conan_list_sin_repetidos.', 'Para el mensaje de odio en la posición', k,' en hate_speech_conan_list_sin_repetidos, el ranking de 3 contranarrativas contiene', metrica1, 'de las', df_contranarrativas_en_conan_para_odio_k.shape[0],'contranarrativas que existen en el conan para ese discurso de odio'))

      # Calculo la métrica_1 para el caso contranarrativa_i, odio_i, odio_k y ranking de 1 contranarrativas 
      # y la guardo en la lista_pares_métrica_1_top1
      df_contranarrativas_en_conan_para_odio_k = sentences_conan.loc[sentences_conan['hateSpeech'] == odio_k, 'counterSpeech'] #fixme: ojo con esto, tengo que chequear que el hate speech que me interesa es efectivamente odio_k.
      metrica1 = 0;
      for m in range(0,len(counternarratives_ranking_list_top1)):
        if counternarratives_ranking_list_top1[m] in df_contranarrativas_en_conan_para_odio_k.values :
            metrica1 += 1;
      lista_pares_métrica_1_top1.append(('Contranarrativa_i, está en la posición', i,'en counternarratives_conan_list_sin_repetidos. Odio_i está en la posición numero', indice_odio_i, 'en hate_speech_conan_list_sin_repetidos.', 'Para el mensaje de odio en la posición', k,' en hate_speech_conan_list_sin_repetidos, el ranking de 3 contranarrativas contiene', metrica1, 'de las', df_contranarrativas_en_conan_para_odio_k.shape[0],'contranarrativas que existen en el conan para ese discurso de odio'))

      #Imprimo estatus de estar corriendo:
      if(iteraciones % 100 == 0):
          print('Calculando métrica 1, iteración número', iteraciones);
      iteraciones += 1;

Calculando métrica 1, iteración número 0
Calculando métrica 1, iteración número 100
Calculando métrica 1, iteración número 200


In [ ]:
#fixme: acá printeo la lista de resultados y veo que hay algunos discursos de odio que tiene muchas contranarrativas 
# (por ejemplo 85), quizás el ranking tenga que ser de más elementos (aunque por otra parte yo quiero que el ranking
# devuelva pocos elementos y muy buenos -tengo el problema de que si el ranking devuelve 10 elementos y los 10 son
# buenos, pero para ese discurso de odio hay 85 contranarrativas, me va a quedar que el sistema devuelve 10 de las 85
# posibles contranarrativas [es un muy mal resultado]-)
# lista_pares_métrica_1_top10


In [72]:
len(lista_pares_métrica_1_top10)

218

#### Guardo los resultados de métrica 1 en archivos (se guardan en la notebook si se corre desde Google Colab y se guardan en la ubicación del script si se corre desde un script .py).

In [73]:
print('Guardando outputs de la métrica 1')

Guardando outputs de la métrica 1


In [74]:
# Voy a guardar el output de la métrica 1 con el ranking de 10 contranarrativas en parte el Dataset en un archivo:
with open(r'metrica1Top10.txt', 'w') as fp:
    for item in lista_pares_métrica_1_top10:
        # write each item on a new line
        fp.write("%s\n" % str(item))
    print('Done')


Done


In [75]:
# Voy a guardar el output de la métrica 1 con el ranking de 5 contranarrativas en parte el Dataset en un archivo:
with open(r'metrica1Top5.txt', 'w') as fp:
    for item in lista_pares_métrica_1_top5:
        # write each item on a new line
        fp.write("%s\n" % str(item))
    print('Done')

Done


In [76]:
# Voy a guardar el output de la métrica 1 con el ranking de 3 contranarrativas en parte el Dataset en un archivo:
with open(r'metrica1Top3.txt', 'w') as fp:
    for item in lista_pares_métrica_1_top3:
        # write each item on a new line
        fp.write("%s\n" % str(item))
    print('Done')

Done


In [77]:
# Voy a guardar el output de la métrica 1 con el ranking de 1 contranarrativas en parte el Dataset en un archivo:
with open(r'metrica1Top1.txt', 'w') as fp:
    for item in lista_pares_métrica_1_top1:
        # write each item on a new line
        fp.write("%s\n" % str(item))
    print('Done')

Done


# Defino metrica_2:

metrica_2 = metrica_1 / min(número de contranarrativas para el un discurso de odio dado, cantidad_elementos_del_ranking).
El mínimo está porque para algunos discursos de odio, el "número de contranarrativas para el un discurso de odio dado" es mucho mayor a "cantidad_elementos_del_ranking" y esto hace parecer que el ranking funciona muy mal, cuando en realidad el tope "cantidad_elementos_del_ranking" hace que no pueda devolver mayor "número de contranarrativas para el un discurso de odio dado".

##Aplico metrica_2:

###Fixme:acá hay muchísimo código repedito, sacar.

In [78]:
print('Calculando métrica 2')

Calculando métrica 2


In [79]:
# Métrica 2 en ranking de 10 elementos
lista_metrica_2_top10 = []
posicion_contranarrativas_encontradas = 8 
posicion_contranarrativas_totales = 10
cantidad_elementos_del_rankingTop10 = 10

for i in range(0, len(lista_pares_métrica_1_top10)):
  metrica_2_para_par_i = lista_pares_métrica_1_top10[i][posicion_contranarrativas_encontradas]/min(lista_pares_métrica_1_top10[i][posicion_contranarrativas_totales],cantidad_elementos_del_rankingTop10)
  lista_metrica_2_top10.append(metrica_2_para_par_i)

In [80]:
# Métrica 2 en ranking de 5 elementos
lista_metrica_2_top5 = []
posicion_contranarrativas_encontradas = 8 
posicion_contranarrativas_totales = 10
cantidad_elementos_del_rankingTop5 = 5

for i in range(0, len(lista_pares_métrica_1_top5)):
  metrica_2_para_par_i = lista_pares_métrica_1_top5[i][posicion_contranarrativas_encontradas]/min(lista_pares_métrica_1_top5[i][posicion_contranarrativas_totales],cantidad_elementos_del_rankingTop5)
  lista_metrica_2_top5.append(metrica_2_para_par_i)

In [81]:
# Métrica 2 en ranking de 3 elementos
lista_metrica_2_top3 = []
posicion_contranarrativas_encontradas = 8 
posicion_contranarrativas_totales = 10
cantidad_elementos_del_rankingTop3 = 3

for i in range(0, len(lista_pares_métrica_1_top3)):
  metrica_2_para_par_i = lista_pares_métrica_1_top3[i][posicion_contranarrativas_encontradas]/min(lista_pares_métrica_1_top3[i][posicion_contranarrativas_totales],cantidad_elementos_del_rankingTop3)
  lista_metrica_2_top3.append(metrica_2_para_par_i)

In [82]:
# Métrica 2 en ranking de 1 elementos
lista_metrica_2_top1 = []
posicion_contranarrativas_encontradas = 8 
posicion_contranarrativas_totales = 10
cantidad_elementos_del_rankingTop1 = 1

for i in range(0, len(lista_pares_métrica_1_top1)):
  metrica_2_para_par_i = lista_pares_métrica_1_top1[i][posicion_contranarrativas_encontradas]/min(lista_pares_métrica_1_top1[i][posicion_contranarrativas_totales],cantidad_elementos_del_rankingTop1)
  lista_metrica_2_top1.append(metrica_2_para_par_i)

#### Guardo los resultados de métrica 2 en archivos (se guardan en la notebook si se corre desde Google Colab y se guardan en la ubicación del script si se corre desde un script .py).

In [83]:
print('Guardando outputs de la métrica 2')

Guardando outputs de la métrica 2


In [84]:
# Voy a guardar el output de la métrica 2 con el ranking de 10 contranarrativas en parte el Dataset en un archivo:
with open(r'metrica2Top10.txt', 'w') as fp:
    for item in lista_metrica_2_top10:
        # write each item on a new line
        fp.write("%s\n" % str(item))
    print('Done')

Done


In [85]:
# Voy a guardar el output de la métrica 2 con el ranking de 5 contranarrativas en parte el Dataset en un archivo:
with open(r'metrica2Top5.txt', 'w') as fp:
    for item in lista_metrica_2_top5:
        # write each item on a new line
        fp.write("%s\n" % str(item))
    print('Done')

Done


In [86]:
# Voy a guardar el output de la métrica 2 con el ranking de 3 contranarrativas en parte el Dataset en un archivo:
with open(r'metrica2Top3.txt', 'w') as fp:
    for item in lista_metrica_2_top3:
        # write each item on a new line
        fp.write("%s\n" % str(item))
    print('Done')

Done


In [87]:
# Voy a guardar el output de la métrica 2 con el ranking de 1 contranarrativas en parte el Dataset en un archivo:
with open(r'metrica2Top1.txt', 'w') as fp:
    for item in lista_metrica_2_top1:
        # write each item on a new line
        fp.write("%s\n" % str(item))
    print('Done')

Done


# Defino metrica_3: promedio de metrica_2:

metrica_3 será el promedio de la metrica_2.

##Aplico metrica_3:

###Fixme:acá hay muchísimo código repedito, sacar.

In [88]:
metrica_3_top10 = sum(lista_metrica_2_top10)/len(lista_metrica_2_top10)
print('Métrica 3, con ranking de 10 elementos:', metrica_3_top10)

Métrica 3, con ranking de 10 elementos: 0.1045871559633027


In [89]:
metrica_3_top5 = sum(lista_metrica_2_top5)/len(lista_metrica_2_top5)
print('Métrica 3, con ranking de 5 elementos:', metrica_3_top5)

Métrica 3, con ranking de 5 elementos: 0.11261467889908251


In [90]:
metrica_3_top3 = sum(lista_metrica_2_top3)/len(lista_metrica_2_top3)
print('Métrica 3, con ranking de 3 elementos:', metrica_3_top3)

Métrica 3, con ranking de 3 elementos: 0.1253822629969419


In [91]:
metrica_3_top1 = sum(lista_metrica_2_top1)/len(lista_metrica_2_top1)
print('Métrica 3, con ranking de 1 elemento:', metrica_3_top1)

Métrica 3, con ranking de 1 elemento: 0.11467889908256881


#### Guardo los resultados de métrica 3 en archivos (se guardan en la notebook si se corre desde Google Colab y se guardan en la ubicación del script si se corre desde un script .py).

In [92]:
print('Guardando outputs de la métrica 3')

Guardando outputs de la métrica 3


#####Fixme:acá hay muchísimo código repedito, sacar.

In [93]:
print('Guardando outputs de la métrica 3')

Guardando outputs de la métrica 3


In [94]:
# Voy a guardar el output de la métrica 3, top 10 en un archivo:
with open(r'metrica3Top10.txt', 'w') as fp:
    fp.write(str(metrica_3_top10))
    print('Done')


Done


In [95]:
# Voy a guardar el output de la métrica 3, top 5 en un archivo:
with open(r'metrica3Top5.txt', 'w') as fp:
    fp.write(str(metrica_3_top5))
    print('Done')

Done


In [96]:
# Voy a guardar el output de la métrica 3, top 3 en un archivo:
with open(r'metrica3Top3.txt', 'w') as fp:
    fp.write(str(metrica_3_top3))
    print('Done')

Done


In [97]:
# Voy a guardar el output de la métrica 3, top 1 en un archivo:
with open(r'metrica3Top1.txt', 'w') as fp:
    fp.write(str(metrica_3_top1))
    print('Done')

Done
